# This NB was created to generate primers for KOing genes in yeast by replacing the gene with KanMX
#with user cloning instead of overlap fragments


In [ ]:
The first part of the NB is the same as USER_Primers: it design user primers to anneal geneUP::KanMX::geneDw together
#the second part uses elements from Yeast KO to design confirmation primers



In [1]:
import numpy as np
import pandas as pd
import  re
from Bio.SeqUtils import MeltingTemp as mt
from Bio.Seq import Seq
from Bio.SeqUtils.MeltingTemp import make_table, DNA_NN2
import pyperclip
import importlib
import DNA_tools as dt

import os
import  Bio

from Bio import SeqIO


#https://biopython.org/docs/1.75/api/Bio.SeqUtils.MeltingTemp.html
Breslauer= mt.DNA_NN1
Santalucia= mt.DNA_NN4 #same as benchling
Tm_Opt =Santalucia

In [2]:
#forward seq of user overhangs:
User_Overhangs={}
User_Overhangs['GV1']= 'CGTGCGAu'
User_Overhangs['GV2']= 'ATCGCGTG'#'CACGCGAu'
User_Overhangs['U1'] = 'AGCCCTCTu'
User_Overhangs['U2'] = 'ACACAGGCu'
User_Overhangs['U3'] = 'ATGCCACCu'
User_Overhangs['U4'] = 'AAGGAGCGu'
User_Overhangs['U5'] = 'ACCGTAGCu'
User_Overhangs['U6'] = 'ACTGCGAGu'
User_Overhangs['U7'] = 'AGTGCAGGu'
User_Overhangs['none'] = 'TATATATATATATA'

In [3]:
#for mads: insertion of BTX in YER084W

YER084W_up={}
YER084W_up['name']='YER084W_CDS'
YER084W_up['seq']='ggtctttgcagcgctaggagcgtctaaaggtgattctgcgttcaaatgactgcttgcctgcccggtgattttattaagcctggagctggcacctccattgctaaacttcttctgtctcctttcccttagtaacctgcgcttttccgcctctgttaattcagacatatttcctagagctacgttgctgatgctacaaaca'
YER084W_up['need_reverse']=0 #if parts needs to be reversed:1 else:0
YER084W_up['Primer_up_U']='GV1' #user site up
YER084W_up['Primer_down_U']='U1'#user site down
YER084W_up['Primer_up']=''
YER084W_up['Primer_up_Tm']=''
YER084W_up['Primer_down']=''
YER084W_up['Primer_down_Tm']=''

BTX={}
BTX['name']='ANTE113_BTX'
BTX['seq']='ATTAAGTCCTCAGCGAGCTCGCATGGAATGCGTGCGATGAGCGACCTCATGCTATACCTGAGAAAGCAACCTGACCTACAGGAAAGAGTTACTCAAGAATAAGAATTTTCGTTTTAAAACCTAAGAGTCACTTTAAAATTTGTATACACTTATTTTTTTTATAACTTATTTAATAATAAAAATCATAAATCATAAGAAATTCGCTTATTTAGAAGTGTCAACAACGTATCTACCAACGGAATGCGTGCGATTTACTCTTCCAACCTTCTTAGCAAGTATTCCACCTCAACTTCCTTAGTGATAGGTATAACAATTTCCTTATAAATTTTTACCAAATATTCTGGAGTAATCCTTCTTTCACCTGACTCGTTGGTAGGGTCCACACCATAGACTTCAGCCTTCTTAGTTAATCTTTCTAGAATCTTTTCTTCAACGGCAGAATTGGTGATATTCTTCATTATCCCCTCGACATCTTTACTTTTGATCAGCTTTGTGTATAGCGGGATATCCGATTGGAACTTGGCTTCAGCAACAAACTTGCCAAAGTGGATTCTCCTACTCAAGCTTTGCAAACATTCTATATCTCTAGTGGCAACAGAACTGAAGTTATTCTTATCATCACCATCTCTTTTCGAAATTAATGGTATAATCTTTTCAATATAAACTTTTTTTATTTTATCATTGTAATTAACTTCTGGGGCATAAGGCGCCAAAATTTGTGGGTAGTTAATGCTCGGTAAGAATGATTTCTGAATCTTGTCAGGAAAGAAGGGAGTTTCATCAGGTGATTCGAATCTTCTGATGCGAGAATGCGCAATTTCAAGATTTGAAAGAGCCCAATCCAAGAAAGATCCTTTAAAATTCGGAATTTCTAAACCTGGATGGTTTGCCTCATAAACTGAAGGACATGTGGCGAAATGCGACCTCTCAATAAATTTGAAGATGATCGAATCCTCCATTCTAACTAATTCATCTCTAATATTTTGTAGATTTAAAACAGTTTCTGGTTTTGTGAAATCCATagacttcgtCCGCTTGTAATTAAAACTTAGATTAGATTGCTATGCTTTCTTTCTAATGAGCAAGAAGTAAAAAAAGTTGTAATAGAACAAGAAAAATGAAACTGAAACTTGAGAAATTGAAGACCGTTTATTAACTTAAATATCAATGGGAGGTCATCGAAAGAGAAAAAAATCAAAAAAAAAAATTTTCAAGAAAAAGAAACGTGATAAAAATTTTTATTGCCTTTTTCGACGAAGAAAAAGAAACGAGGCGGTCTCTTTTTTCTTTTCCAAACCTTTAGTACGGGTAATTAACGACACCCTAGAGGAAGAAAGAGGGGAAATTTAGTATGCTGTGCTTGGGTGTTTTGAAGTGGTACGGCGATGCGCGGAGTCCGAGAAAATCTGGAAGAGTAAAAAAGGAGTAGAAACATTTTGAAGCTATGGTGTGTGCGGCCGGCCTGGAAGTACCTTCAAAGAATGGGGTCTTATCTTGTTTTGCAAGTACCACTGAGCAGGATAATAATAGAAATGATAATATACTATAGTAGAGATAACGTCGATGACTTCCCATACTGTAATTGCTTTTAGTTGTGTATTTTTAGTGTGCAAGTTTCTGTAAATCGATTAATTTTTTTTTCTTTCCTCTTTTTATTAACCTTAATTTTTATTTTAGATTCCTGACTTCAACTCAAGACGCACAGATATTATAACATCTGCATAATAGGCATTTGCAAGAATTACTCGTGAGTAAGGAAAGAGTGAGGAACTATCGCATACCTGCATTTAAAGATGCCGATTTGGGCGCGAATCCTTTATTTTGGCTTCACCCTCATACTATTATCAGGGCCAGAAAAAGGAAGTGTTTCCCTCCTTCTTGAATTGATGTTACCCTCATAAAGCACGTGGCCTCTTATCGAGAAAGAAATTACCGTCGCTCGTGATTTGTTTGCAAAAAGAACAAAACTGAAAAAACCCAGACACGCTCGACTTCCTGTCTTCCTATTGATTGCAGCTTCCAATTTCGTCACACAACAAGGTCCTAGCGACGGCTCACAGGTTTTGTAACAAGCAATCGAAGGTTCTGGAATGGCGGGAAAGGGTTTAGTACCACATGCTATGATGCCCACTGTGATCTCCAGAGCAAAGTTCGTTCGATCGTACTGTTACTCTCTCTCTTTCAAACAGAATTGTCCGAATCGTGTGACAACAACAGCCTGTTCTCACACACTCTTTTCTTCTAACCAAGGGGGTGGTTTAGTTTAGTAGAACCTCGTGAAACTTACATTTACATATATATAAACTTGCATAAATTGGTCAATGCAAGAAATACATATTTGGTCTTTTCTAATTCGTAGTTTTTCAAGTTCTTAGATGCTTTCTTTTTCTCTTTTTTACAGATCATCAAGGAAGTAATTATCTACTTTTTACAACAAATATAAAACAAGGATCCaagtctactATGAGTGAATCTCCAATGTTCGCTGCCAACGGCATGCCAAAGGTAAATCAAGGTGCTGAAGAAGATGTCAGAATTTTAGGTTACGACCCATTAGCTTCTCCAGCTCTCCTTCAAGTGCAAATCCCAGCCACACCAACTTCTTTGGAAACTGCCAAGAGAGGTAGAAGAGAAGCTATAGATATTATTACCGGTAAAGACGACAGAGTTCTTGTCATTGTCGGTCCTTGTTCCATCCATGATCTAGAAGCCGCTCAAGAATACGCTTTGAGATTAAAGAAATTGTCAGATGAATTAAAAGGTGATTTATCCATCATTATGAGAGCATACTTGGAGAAGCCAAGAACAACCGTCGGCTGGAAAGGTCTAATTAATGACCCTGATGTTAACAACACTTTCAACATCAACAAGGGTTTGCAATCCGCTAGACAATTGTTTGTCAACTTGACAAATATCGGTTTGCCAATTGGTTCTGAAATGCTTGATACCATTTCTCCTCAATACTTGGCTGATTTGGTCTCCTTCGGTGCCATTGGTGCCAGAACCACCGAATCTCAACTGCACAGAGAATTGGCCTCCGGTTTGTCTTTCCCAGTTGGTTTCAAGAACGGTACCGATGGTACCTTAAATGTTGCTGTGGATGCTTGTCAAGCCGCTGCTCATTCTCACCATTTCATGGGTGTTACTTTACATGGTGTTGCTGCTATCACCACTACTAAGGGTAACGAACACTGCTTCGTTATTCTAAGAGGTGGTAAAAAGGGTACCAACTACGACGCTAAGTCCGTTGCAGAAGCTAAGGCTCAATTGCCTGCCGGTTCCAACGGTCTAATGATTGACTACTCTCACGGTAACTCCAATAAGGATTTCAGAAACCAACCAAAGGTCAATGACGTTGTTTGTGAGCAAATCGCTAACGGTGAAAACGCCATTACCGGTGTCATGATTGAATCAAACATCAACGAAGGTAACCAAGGCATCCCAGCCGAAGGTAAAGCCGGCTTGAAATATGGTGTTTCCATCACTGATGCTTGTATAGGTTGGGAAACTACTGAAGACGTCTTGAGGAAATTGGCTGCTGCTGTCAGACAAAGAAGAGAAGTTAACAAGAAATAGATTCTGCAGGTAGGGAAAGATATGAGCTATACAGCGGAATTTCCATATCACTCAGATTTTGTTATCTAATTTTTTCCTTCCCACGTCCGCGGGAATCTGTGTATATTACTGCATCTAGATATATGTTATCTTATCTTGGCGCGTACATTTAATTTTCAACGTATTCTATAAGAAATTGCGGGAGTTTTTTTCATGTAGATGATACTGACTGCACGCAAATATAGGCATGATTTATAGGCATGATTTGATGGCTGTACCGATAGGAACGCTAAGAGTAACTTCAGAATCGTTATCCTGGCGGAAAAAATTCATTTGTAAACTTTAAAAAAAAAAGCCAATATCCCCAAAATTATTAAGAGCGCCTCCATTATTAACTAAAATTTCACTCAGCATCCACAATGTATCAGGTATCTACTACAGATATTACATGTGGCGAAAAAGACAAGAACAATGCAATAGCGCATCAAGAAAAAACACAAAGCTTTCAATCAATGAATCGAAAATGTCATTAAAATAGTATATAAATTGAAACTAAGTCATAAAGCTATAAAAAGAAAATTTATTTAAATGCAAGATTTAAAGTAAATTCACTTAGGATCCGTATCTTGGAATTGGGATCAATTTTAAAGGTTTAGTTTTCTGCAAGGCTATACCAAACTTTTCATCCATGTCCAAATCTTTAGGTGAAATGTCTCCCTCTAATTTCCAGTTGAAGAATTGCAATAATGTGGCCAACATCAATGTCAACATCCTTATGGCCAAATTCATACCAGGGCAAATTCTTCTACCAGCACCAAAAGGCAATAAACCAAAATCTCTTCCCTTTACATCAATCTCACAACCTATAAATCTTTCAGGTGAGAAAATATCGGCATTTTGCCAAGCATTTGGATCTCTACCTATGGCCCACAAGTTTACCAAAATTTGAGCATCCTTTGGAACAATGTATCCGTACAATTCTACGTCTGTGTCGGCTTTTCTAGGCAACAAGAAAACTGTAGGTGGGTGCAACCTCAAAGTTTCCTTAATGATTGCTTGCAAGTATGGCAAATTTATTATGTCTGACTCTTGGATTTGCTTATCCTTACCCAATACCTGTTTTATTTCCTCTTGGGCCTTTTCCATCATTTCAGGGTTTCTTATTAACTCAGTCATAACCCACTCAAAGGTAGATGATGTAGTATCAGTACCGGCATCGAAAATATCGACTAACAAGTGGTTAATCTCTCCCATAGTTAATTCATTCTGCTTAAACAATTGCAATAATACGTCCAAGACATCATCTGTAGTTGTAGTAGTTGTAGTTGATGAATCTGGAGCTAACCTCTCACATATTATTCCTTGAAATACTGCAATTAACTTATCAAAAGAACATGCCAACCTTCTTCTAATTCCTGATGGATCGACGCAACCTAAAATTGGGAAGTAATCGGCGTAGTTTGGTTTACCGATGTCTTCCATAATATTCCAAATTAATTCCTTAAACTCTTGAGAGGTATGTGATTTATGATGGGCCAATTCAACAGAAAAGAATAATTTAGACAATAAATTCAAAGAAGTTGTGAAGGCAGCTTTACCAATGTCAACTGCCTGACCCTTCTGAGCGCACTCTTGAACGTATTCATATAATTGCTGAACCTTTGCATGTCTAAAGGTCTGGCATGCATCTAATCTTTGTGGAGACAATAAATGAACAGCAGTTATCTTTCTGAAGTTTCTCCATTTAGGTGAAACAGGCAACCAAGACATAGTCAATTTGTGGTGGTCACCAGCAGTTACTGAATTTGGAATTGTCCTGTTTGATAAAGGGTGATCCTTCTTCAAGAACATCTCTTTGGCGACATCAGCAGATGATACAACAATGGTGGTAACTGATCCTAACCTCAAAGAGATTAAAGGACCGTGGATCTTAGCCAAATTTGCAAATGATCTATGAGGCTTCTTTCCTACTTCCAAGATATTACCAATAATTGGTAAAGGTTTAGGACCTGGAGGTAACAATTTTGTTGTTTGTTGTGAGAATAATAATTTAATAAAATGAAATGAAATAAACCAAATTGCTAATATCATAGCTAATGTTGCATGATCCATTTTTAGTTTATGTATGTGTTTTTTGTAGTTATAGATTTAAGCAAGAAAAGAATACAAACAAAAAATTGAAAAAGATTGATTTAGAATTAAAAAGAAAAATATTTACGTAAGAAGGGAAAATAGTAAATGTTGCAAGTTCACTAAACTCCTAAATTATGCTGCCCTTTATATTCCCTGTTACAGCAGCCGAGCCAAAGGTATATAGGCTCCTTTGCATTAGCATGCGTAACAAACCACCTGTCAGTTTCAACCGAGGTGGTATCCGAGAGAATTGTGTGATTGCTTTAATTAATTTCGGAGAATCTCACATGCCACTGAAGATTAAAAACTGGATGCCAGAAAAGGGGTGTCCAGGTGTAACATCAATAGAGGAAGCTGAAAAGTCTTAGAACGGGTAATCTTCCACCAACCTGATGGGTTCCTAGATATAATCTCGAAGGGAATAAGTAGGGTGATACCGCAGAAGTGTCTGAATGTATTAAGGTCCTCACAGTTTAAATCCCGCTCACACTAACGTAGGATTATTATAACTCAAAAAAATGGCATTATTCTAAGTAAGTTAAATATCCGTAATCTTTAAACAagacttcgTCACCCATGAACCACACGGTTAGTCCAAAAGGGGCAGTTCAGATTCCAGATGCGGGAATTAGCTTGCTGCCACCCTCACCTCACTAACGCTGCGGTGTGCGGATACTTCATGCTATTTATAGACGCGCGTGTCGGAATCAGCACGCGCAAGAACCAAATGGGAAAATCGGAATGGGTCCAGAACTGCTTTGAGTGCTGGCTATTGGCGTCTGATTTCCGTTTTGGGAATCCTTTGCCGCGCGCCCCTCTCAAAACTCCGCACAAGTCCCAGAAAGCGGGAAAGAAATAAAACGCCACCAAAAAAAAAAAAATAAAAGCCAATCCTCGAAGCGTGGGTGGTAGGCCCTGGATTATCCCGTACAAGTATTTCTCAGGAGTAAAAAAACCGTTTGTTTTGGAATTTCCCATTTCGCGGCCACCTACGCCGCTATCTTTGCAACAACTATCTGCGATAACTCAGCAAATTTTGCATATTCGTGTTGCAGTATTGCGATAATGGGAGTCTTACTTCCAACATAACGGCAGAAAGAAATGTGAGAAAATTTTGCATCCTTTGCCTCCGTTCAAGTATATAAAGTCGGCATGCTTGATAATCTTTCTTTCCATCCTACATTGTTCTAATTATTCTTATTCTCCTTTATTCTTTCCTAACATACCAAGAAATTAATCTTCTGTCATTCGCTTAAACACTATATCAATAAAGATCTAAAAATGAAGGGAACCTACTACATCAACCACGGTGACCCATTGATGTACTTGAAGAAGCACATCAAGTTGAGACAATTCTTGGAAGGTTGGCAAGAAAACGTTGTTATCGAAAAGCCAAAGTCTATCTTGATCATCAGTGCACACTGGGACACCAACGTTCCAACCGTTAACTTCGTTGAACACTGTGACACCATCCACGACTTCGACGACTACCCAGACCCATTGTACCAAATCCAATACCGCGCTCCAGGTGCTCCAAACTTGGCTAAGAAGGTTGAAGAATTGCTTAAGGAATCTGGTATGGAATGTGAAATCGACACCAAGAGAGGTTTGGACCACGCTGCTTGGTTCCCATTGATGTTCATGTACCCAGAAGCTAACATCCCAATCTGTGAATTGTCTGTTCAACCATCTAAGGACGGTATCCACCACTACAACGTTGGTAAAGCTTTGTCTCCATTGTTGCAACAAGGTGTTTTGATCATCGGTTCTGGTGGAACCGTTCACCCATCTGACGACACCCCACACTGTCCAAACGGTGTTGCTCCATGGGCTATCGAGTTCGACAACTGGTTGGAAGACGCTTTGCTCTCGGGTAGATACGAAGACGTTAACAACTTCAAGAAGTTGGCTCCAAACTGGGAAATCTCTCACCCAGGTCAAGAACACTTGTACCCATTGCACGTTGCTTTGGGTGCTGCTGGTAAGAACCCAAAGACCCAATTGATCCACCGGTCTTGGGCTGCTAACGGTGTTTTCGGTTACTCTACCTACAACTTCACCCCAACCACCCAAAAGACCGACGGATCCTAAATCGCGTGCATTCATCCGCTCTAACCGAAAAGGAAGGAGTTAGACAACCTGAAGTCTAGGTCCCTATTTATTTTTTTATAGTTATGTTAGTATTAAGAACGTTATTTATATTTCAAATTTTTCTTTTTTTTCTGTACAGACGCGTGTACGCATGTAACATTATACTGAAAACCTTGCTTGAGAAGGTTTTGGGACGCTCGAAGATCGCGTCAGCTGAAGCTTCGTACGCTGCAGGTCGACAATTCTGCAGGTCGACAACCCTTAATATAACTTCGTATAATGTATGCTATACGAAGTTATTAGGTCTAGAGATCTGTTTAGCTTGCCTCGTCCCCGCCGGGTCACCCGGCCAGCGACATGGAGGCCCAGAATACCCTCCTTGACAGTCTTGACGTGCGCAGCTCAGGGGCATGATGTGACTGTCGCCCGTACATTTAGCCCATACATCCCCATGTATAATCATTTGCATCCATACATTTTGATGGCCGCACGGCGCGAAGCAAAAATTACGGCTCCTCGCTGCAGACCTGCGAGCAGGGAAACGCTCCCCTCACAGACGCGTTGAATTGTCCCCACGCCGCGCCCCTGTAGAGAAATATAAAAGGTTAGGATTTGCCACTGAGGTTCTTCTTTCATATACTTCCTTTTAAAATCTTGCTAGGATACAGTTCTCACATCACATCCGAACATAAACAACCATGGGTACCACTCTTGACGACACGGCTTACCGGTACCGCACCAGTGTCCCGGGGGACGCCGAGGCCATCGAGGCACTGGATGGGTCCTTCACCACCGACACCGTCTTCCGCGTCACCGCCACCGGGGACGGCTTCACCCTGCGGGAGGTGCCGGTGGACCCGCCCCTGACCAAGGTGTTCCCCGACGACGAATCGGACGACGAATCGGACGACGGGGAGGACGGCGACCCGGACTCCCGGACGTTCGTCGCGTACGGGGACGACGGCGACCTGGCGGGCTTCGTGGTCGTCTCGTACTCCGGCTGGAACCGCCGGCTGACCGTCGAGGACATCGAGGTCGCCCCGGAGCACCGGGGGCACGGGGTCGGGCGCGCGTTGATGGGGCTCGCGACGGAGTTCGCCCGCGAGCGGGGCGCCGGGCACCTCTGGCTGGAGGTCACCAACGTCAACGCACCGGCGATCCACGCGTACCGGCGGATGGGGTTCACCCTCTGCGGCCTGGACACCGCCCTGTACGACGGCACCGCCTCGGACGGCGAGCAGGCGCTCTACATGAGCATGCCCTGCCCCTAATCAGTACTGACAATAAAAAGATTCTTGTTTTCAAGAACTTGTCATTTGTATAGTTTTTTTATATTGTAGTTGTTCTATTTTAATCAAATGTTAGCGTGATTTATATTTTTTTTCGCCTCGACATCATCTGCCCAGATGCGAAGTTAAGTGCGCAGAAAGTAATATCATGCGTCAATCGTATGTGAATGCTGGTCGCTATACTGCTGTCGATTCGATACTAACGCCGCCATCCAGTGTCGAAAACGAGCTCTCGAGAACCCTTAATATAACTTCGTATAATGTATGCTATACGAAGTTATTAGGTGATATCAGATCCACTAGTG'
BTX['need_reverse']=0 #if parts needs to be reversed:1 else:0
BTX['Primer_up_U']='U1' #user site up
BTX['Primer_down_U']='U2'#user site down
BTX['Primer_up']=''
BTX['Primer_up_Tm']=''
BTX['Primer_down']=''
BTX['Primer_down_Tm']=''

YER084W_dw={}
YER084W_dw['name']='YER084W_CDS'
YER084W_dw['seq']='taactcagattttcaagaaacagaatagcacattctctaacatcaagttatcataccaatatataacacaaattatgcataaatagataagtaatatgtccctctgcccctgcctttccctttccagatgctggcatcgaaagtaaggcatgtatttgtatgtaagttttttaattaatagtcttcttctttcttatcag'
YER084W_dw['need_reverse']=0 #if parts needs to be reversed:1 else:0
YER084W_dw['Primer_up_U']='U2' #user site up
YER084W_dw['Primer_down_U']='GV2'#user site down
YER084W_dw['Primer_up']=''
YER084W_dw['Primer_up_Tm']=''
YER084W_dw['Primer_down']=''
YER084W_dw['Primer_down_Tm']=''

YER084W_KO_BTX=[YER084W_up,BTX,YER084W_dw]

In [122]:
### Design the primers for addind CDSs to the BTX pathway + making the KO:
###up regions (promoter): 1000bp upstream of CDS unless another sequeunce is there
### upstream is 500bp regardless of presence of other CDS (might include the end of another gene)
### for PDX3 promoter is close to snoRNA, so only took 240bp long promoter
### for PDX3 terminator is close to start of other CDS so took a shorter terminator
##down region (terminator): 500dp dwnstream of CDS unless another sequeunce is there

#for the KO primers (reverse of up and forward of down) to avoid nesting promoter>>promoter::KANMX::terminator>>terminator
#end up region so that total length is 200bp
#start dwn region so that total length is 200bp

#YDL119C =HEM25
#KGD2 v
#PPA1 = VMA16
#CAX4
#Pro2
#GLR1
#YSY6
#PDX3

#BTX pathway will be annealed straight into pCCM021 to be flanked by XII-up and  HomoB
BTX={}
BTX['name']='ANTE113_BTX'
BTX['seq']='GCGACCTCATGCTATACCTGAGAAAGCAACCTGACCTACAGGAAAGAGTTACTCAAGAATAAGAATTTTCGTTTTAAAACCTAAGAGTCACTTTAAAATTTGTATACACTTATTTTTTTTATAACTTATTTAATAATAAAAATCATAAATCATAAGAAATTCGCTTATTTAGAAGTGTCAACAACGTATCTACCAACGGAATGCGTGCGATTTACTCTTCCAACCTTCTTAGCAAGTATTCCACCTCAACTTCCTTAGTGATAGGTATAACAATTTCCTTATAAATTTTTACCAAATATTCTGGAGTAATCCTTCTTTCACCTGACTCGTTGGTAGGGTCCACACCATAGACTTCAGCCTTCTTAGTTAATCTTTCTAGAATCTTTTCTTCAACGGCAGAATTGGTGATATTCTTCATTATCCCCTCGACATCTTTACTTTTGATCAGCTTTGTGTATAGCGGGATATCCGATTGGAACTTGGCTTCAGCAACAAACTTGCCAAAGTGGATTCTCCTACTCAAGCTTTGCAAACATTCTATATCTCTAGTGGCAACAGAACTGAAGTTATTCTTATCATCACCATCTCTTTTCGAAATTAATGGTATAATCTTTTCAATATAAACTTTTTTTATTTTATCATTGTAATTAACTTCTGGGGCATAAGGCGCCAAAATTTGTGGGTAGTTAATGCTCGGTAAGAATGATTTCTGAATCTTGTCAGGAAAGAAGGGAGTTTCATCAGGTGATTCGAATCTTCTGATGCGAGAATGCGCAATTTCAAGATTTGAAAGAGCCCAATCCAAGAAAGATCCTTTAAAATTCGGAATTTCTAAACCTGGATGGTTTGCCTCATAAACTGAAGGACATGTGGCGAAATGCGACCTCTCAATAAATTTGAAGATGATCGAATCCTCCATTCTAACTAATTCATCTCTAATATTTTGTAGATTTAAAACAGTTTCTGGTTTTGTGAAATCCATagacttcgtCCGCTTGTAATTAAAACTTAGATTAGATTGCTATGCTTTCTTTCTAATGAGCAAGAAGTAAAAAAAGTTGTAATAGAACAAGAAAAATGAAACTGAAACTTGAGAAATTGAAGACCGTTTATTAACTTAAATATCAATGGGAGGTCATCGAAAGAGAAAAAAATCAAAAAAAAAAATTTTCAAGAAAAAGAAACGTGATAAAAATTTTTATTGCCTTTTTCGACGAAGAAAAAGAAACGAGGCGGTCTCTTTTTTCTTTTCCAAACCTTTAGTACGGGTAATTAACGACACCCTAGAGGAAGAAAGAGGGGAAATTTAGTATGCTGTGCTTGGGTGTTTTGAAGTGGTACGGCGATGCGCGGAGTCCGAGAAAATCTGGAAGAGTAAAAAAGGAGTAGAAACATTTTGAAGCTATGGTGTGTGCGGCCGGCCTGGAAGTACCTTCAAAGAATGGGGTCTTATCTTGTTTTGCAAGTACCACTGAGCAGGATAATAATAGAAATGATAATATACTATAGTAGAGATAACGTCGATGACTTCCCATACTGTAATTGCTTTTAGTTGTGTATTTTTAGTGTGCAAGTTTCTGTAAATCGATTAATTTTTTTTTCTTTCCTCTTTTTATTAACCTTAATTTTTATTTTAGATTCCTGACTTCAACTCAAGACGCACAGATATTATAACATCTGCATAATAGGCATTTGCAAGAATTACTCGTGAGTAAGGAAAGAGTGAGGAACTATCGCATACCTGCATTTAAAGATGCCGATTTGGGCGCGAATCCTTTATTTTGGCTTCACCCTCATACTATTATCAGGGCCAGAAAAAGGAAGTGTTTCCCTCCTTCTTGAATTGATGTTACCCTCATAAAGCACGTGGCCTCTTATCGAGAAAGAAATTACCGTCGCTCGTGATTTGTTTGCAAAAAGAACAAAACTGAAAAAACCCAGACACGCTCGACTTCCTGTCTTCCTATTGATTGCAGCTTCCAATTTCGTCACACAACAAGGTCCTAGCGACGGCTCACAGGTTTTGTAACAAGCAATCGAAGGTTCTGGAATGGCGGGAAAGGGTTTAGTACCACATGCTATGATGCCCACTGTGATCTCCAGAGCAAAGTTCGTTCGATCGTACTGTTACTCTCTCTCTTTCAAACAGAATTGTCCGAATCGTGTGACAACAACAGCCTGTTCTCACACACTCTTTTCTTCTAACCAAGGGGGTGGTTTAGTTTAGTAGAACCTCGTGAAACTTACATTTACATATATATAAACTTGCATAAATTGGTCAATGCAAGAAATACATATTTGGTCTTTTCTAATTCGTAGTTTTTCAAGTTCTTAGATGCTTTCTTTTTCTCTTTTTTACAGATCATCAAGGAAGTAATTATCTACTTTTTACAACAAATATAAAACAAGGATCCaagtctactATGAGTGAATCTCCAATGTTCGCTGCCAACGGCATGCCAAAGGTAAATCAAGGTGCTGAAGAAGATGTCAGAATTTTAGGTTACGACCCATTAGCTTCTCCAGCTCTCCTTCAAGTGCAAATCCCAGCCACACCAACTTCTTTGGAAACTGCCAAGAGAGGTAGAAGAGAAGCTATAGATATTATTACCGGTAAAGACGACAGAGTTCTTGTCATTGTCGGTCCTTGTTCCATCCATGATCTAGAAGCCGCTCAAGAATACGCTTTGAGATTAAAGAAATTGTCAGATGAATTAAAAGGTGATTTATCCATCATTATGAGAGCATACTTGGAGAAGCCAAGAACAACCGTCGGCTGGAAAGGTCTAATTAATGACCCTGATGTTAACAACACTTTCAACATCAACAAGGGTTTGCAATCCGCTAGACAATTGTTTGTCAACTTGACAAATATCGGTTTGCCAATTGGTTCTGAAATGCTTGATACCATTTCTCCTCAATACTTGGCTGATTTGGTCTCCTTCGGTGCCATTGGTGCCAGAACCACCGAATCTCAACTGCACAGAGAATTGGCCTCCGGTTTGTCTTTCCCAGTTGGTTTCAAGAACGGTACCGATGGTACCTTAAATGTTGCTGTGGATGCTTGTCAAGCCGCTGCTCATTCTCACCATTTCATGGGTGTTACTTTACATGGTGTTGCTGCTATCACCACTACTAAGGGTAACGAACACTGCTTCGTTATTCTAAGAGGTGGTAAAAAGGGTACCAACTACGACGCTAAGTCCGTTGCAGAAGCTAAGGCTCAATTGCCTGCCGGTTCCAACGGTCTAATGATTGACTACTCTCACGGTAACTCCAATAAGGATTTCAGAAACCAACCAAAGGTCAATGACGTTGTTTGTGAGCAAATCGCTAACGGTGAAAACGCCATTACCGGTGTCATGATTGAATCAAACATCAACGAAGGTAACCAAGGCATCCCAGCCGAAGGTAAAGCCGGCTTGAAATATGGTGTTTCCATCACTGATGCTTGTATAGGTTGGGAAACTACTGAAGACGTCTTGAGGAAATTGGCTGCTGCTGTCAGACAAAGAAGAGAAGTTAACAAGAAATAGATTCTGCAGGTAGGGAAAGATATGAGCTATACAGCGGAATTTCCATATCACTCAGATTTTGTTATCTAATTTTTTCCTTCCCACGTCCGCGGGAATCTGTGTATATTACTGCATCTAGATATATGTTATCTTATCTTGGCGCGTACATTTAATTTTCAACGTATTCTATAAGAAATTGCGGGAGTTTTTTTCATGTAGATGATACTGACTGCACGCAAATATAGGCATGATTTATAGGCATGATTTGATGGCTGTACCGATAGGAACGCTAAGAGTAACTTCAGAATCGTTATCCTGGCGGAAAAAATTCATTTGTAAACTTTAAAAAAAAAAGCCAATATCCCCAAAATTATTAAGAGCGCCTCCATTATTAACTAAAATTTCACTCAGCATCCACAATGTATCAGGTATCTACTACAGATATTACATGTGGCGAAAAAGACAAGAACAATGCAATAGCGCATCAAGAAAAAACACAAAGCTTTCAATCAATGAATCGAAAATGTCATTAAAATAGTATATAAATTGAAACTAAGTCATAAAGCTATAAAAAGAAAATTTATTTAAATGCAAGATTTAAAGTAAATTCACTTAGGATCCGTATCTTGGAATTGGGATCAATTTTAAAGGTTTAGTTTTCTGCAAGGCTATACCAAACTTTTCATCCATGTCCAAATCTTTAGGTGAAATGTCTCCCTCTAATTTCCAGTTGAAGAATTGCAATAATGTGGCCAACATCAATGTCAACATCCTTATGGCCAAATTCATACCAGGGCAAATTCTTCTACCAGCACCAAAAGGCAATAAACCAAAATCTCTTCCCTTTACATCAATCTCACAACCTATAAATCTTTCAGGTGAGAAAATATCGGCATTTTGCCAAGCATTTGGATCTCTACCTATGGCCCACAAGTTTACCAAAATTTGAGCATCCTTTGGAACAATGTATCCGTACAATTCTACGTCTGTGTCGGCTTTTCTAGGCAACAAGAAAACTGTAGGTGGGTGCAACCTCAAAGTTTCCTTAATGATTGCTTGCAAGTATGGCAAATTTATTATGTCTGACTCTTGGATTTGCTTATCCTTACCCAATACCTGTTTTATTTCCTCTTGGGCCTTTTCCATCATTTCAGGGTTTCTTATTAACTCAGTCATAACCCACTCAAAGGTAGATGATGTAGTATCAGTACCGGCATCGAAAATATCGACTAACAAGTGGTTAATCTCTCCCATAGTTAATTCATTCTGCTTAAACAATTGCAATAATACGTCCAAGACATCATCTGTAGTTGTAGTAGTTGTAGTTGATGAATCTGGAGCTAACCTCTCACATATTATTCCTTGAAATACTGCAATTAACTTATCAAAAGAACATGCCAACCTTCTTCTAATTCCTGATGGATCGACGCAACCTAAAATTGGGAAGTAATCGGCGTAGTTTGGTTTACCGATGTCTTCCATAATATTCCAAATTAATTCCTTAAACTCTTGAGAGGTATGTGATTTATGATGGGCCAATTCAACAGAAAAGAATAATTTAGACAATAAATTCAAAGAAGTTGTGAAGGCAGCTTTACCAATGTCAACTGCCTGACCCTTCTGAGCGCACTCTTGAACGTATTCATATAATTGCTGAACCTTTGCATGTCTAAAGGTCTGGCATGCATCTAATCTTTGTGGAGACAATAAATGAACAGCAGTTATCTTTCTGAAGTTTCTCCATTTAGGTGAAACAGGCAACCAAGACATAGTCAATTTGTGGTGGTCACCAGCAGTTACTGAATTTGGAATTGTCCTGTTTGATAAAGGGTGATCCTTCTTCAAGAACATCTCTTTGGCGACATCAGCAGATGATACAACAATGGTGGTAACTGATCCTAACCTCAAAGAGATTAAAGGACCGTGGATCTTAGCCAAATTTGCAAATGATCTATGAGGCTTCTTTCCTACTTCCAAGATATTACCAATAATTGGTAAAGGTTTAGGACCTGGAGGTAACAATTTTGTTGTTTGTTGTGAGAATAATAATTTAATAAAATGAAATGAAATAAACCAAATTGCTAATATCATAGCTAATGTTGCATGATCCATTTTTAGTTTATGTATGTGTTTTTTGTAGTTATAGATTTAAGCAAGAAAAGAATACAAACAAAAAATTGAAAAAGATTGATTTAGAATTAAAAAGAAAAATATTTACGTAAGAAGGGAAAATAGTAAATGTTGCAAGTTCACTAAACTCCTAAATTATGCTGCCCTTTATATTCCCTGTTACAGCAGCCGAGCCAAAGGTATATAGGCTCCTTTGCATTAGCATGCGTAACAAACCACCTGTCAGTTTCAACCGAGGTGGTATCCGAGAGAATTGTGTGATTGCTTTAATTAATTTCGGAGAATCTCACATGCCACTGAAGATTAAAAACTGGATGCCAGAAAAGGGGTGTCCAGGTGTAACATCAATAGAGGAAGCTGAAAAGTCTTAGAACGGGTAATCTTCCACCAACCTGATGGGTTCCTAGATATAATCTCGAAGGGAATAAGTAGGGTGATACCGCAGAAGTGTCTGAATGTATTAAGGTCCTCACAGTTTAAATCCCGCTCACACTAACGTAGGATTATTATAACTCAAAAAAATGGCATTATTCTAAGTAAGTTAAATATCCGTAATCTTTAAACAagacttcgTCACCCATGAACCACACGGTTAGTCCAAAAGGGGCAGTTCAGATTCCAGATGCGGGAATTAGCTTGCTGCCACCCTCACCTCACTAACGCTGCGGTGTGCGGATACTTCATGCTATTTATAGACGCGCGTGTCGGAATCAGCACGCGCAAGAACCAAATGGGAAAATCGGAATGGGTCCAGAACTGCTTTGAGTGCTGGCTATTGGCGTCTGATTTCCGTTTTGGGAATCCTTTGCCGCGCGCCCCTCTCAAAACTCCGCACAAGTCCCAGAAAGCGGGAAAGAAATAAAACGCCACCAAAAAAAAAAAAATAAAAGCCAATCCTCGAAGCGTGGGTGGTAGGCCCTGGATTATCCCGTACAAGTATTTCTCAGGAGTAAAAAAACCGTTTGTTTTGGAATTTCCCATTTCGCGGCCACCTACGCCGCTATCTTTGCAACAACTATCTGCGATAACTCAGCAAATTTTGCATATTCGTGTTGCAGTATTGCGATAATGGGAGTCTTACTTCCAACATAACGGCAGAAAGAAATGTGAGAAAATTTTGCATCCTTTGCCTCCGTTCAAGTATATAAAGTCGGCATGCTTGATAATCTTTCTTTCCATCCTACATTGTTCTAATTATTCTTATTCTCCTTTATTCTTTCCTAACATACCAAGAAATTAATCTTCTGTCATTCGCTTAAACACTATATCAATAAAGATCTAAAAATGAAGGGAACCTACTACATCAACCACGGTGACCCATTGATGTACTTGAAGAAGCACATCAAGTTGAGACAATTCTTGGAAGGTTGGCAAGAAAACGTTGTTATCGAAAAGCCAAAGTCTATCTTGATCATCAGTGCACACTGGGACACCAACGTTCCAACCGTTAACTTCGTTGAACACTGTGACACCATCCACGACTTCGACGACTACCCAGACCCATTGTACCAAATCCAATACCGCGCTCCAGGTGCTCCAAACTTGGCTAAGAAGGTTGAAGAATTGCTTAAGGAATCTGGTATGGAATGTGAAATCGACACCAAGAGAGGTTTGGACCACGCTGCTTGGTTCCCATTGATGTTCATGTACCCAGAAGCTAACATCCCAATCTGTGAATTGTCTGTTCAACCATCTAAGGACGGTATCCACCACTACAACGTTGGTAAAGCTTTGTCTCCATTGTTGCAACAAGGTGTTTTGATCATCGGTTCTGGTGGAACCGTTCACCCATCTGACGACACCCCACACTGTCCAAACGGTGTTGCTCCATGGGCTATCGAGTTCGACAACTGGTTGGAAGACGCTTTGCTCTCGGGTAGATACGAAGACGTTAACAACTTCAAGAAGTTGGCTCCAAACTGGGAAATCTCTCACCCAGGTCAAGAACACTTGTACCCATTGCACGTTGCTTTGGGTGCTGCTGGTAAGAACCCAAAGACCCAATTGATCCACCGGTCTTGGGCTGCTAACGGTGTTTTCGGTTACTCTACCTACAACTTCACCCCAACCACCCAAAAGACCGACGGATCCTAAATCGCGTGCATTCATCCGCTCTAACCGAAAAGGAAGGAGTTAGACAACCTGAAGTCTAGGTCCCTATTTATTTTTTTATAGTTATGTTAGTATTAAGAACGTTATTTATATTTCAAATTTTTCTTTTTTTTCTGTACAGACGCGTGTACGCATGTAACATTATACTGAAAACCTTGCTTGAGAAGGTTTTGGGACGCTCGAAGATCGCGTCAGCTGAAGCTTCGTACGCTGCAGGTCGACAATTCTGCAGGTCGACAACCCTTAATATAACTTCGTATAATGTATGCTATACGAAGTTATTAGGTCTAGAGATCTGTTTAGCTTGCCTCGTCCCCGCCGGGTCACCCGGCCAGCGACATGGAGGCCCAGAATACCCTCCTTGACAGTCTTGACGTGCGCAGCTCAGGGGCATGATGTGACTGTCGCCCGTACATTTAGCCCATACATCCCCATGTATAATCATTTGCATCCATACATTTTGATGGCCGCACGGCGCGAAGCAAAAATTACGGCTCCTCGCTGCAGACCTGCGAGCAGGGAAACGCTCCCCTCACAGACGCGTTGAATTGTCCCCACGCCGCGCCCCTGTAGAGAAATATAAAAGGTTAGGATTTGCCACTGAGGTTCTTCTTTCATATACTTCCTTTTAAAATCTTGCTAGGATACAGTTCTCACATCACATCCGAACATAAACAACCATGGGTACCACTCTTGACGACACGGCTTACCGGTACCGCACCAGTGTCCCGGGGGACGCCGAGGCCATCGAGGCACTGGATGGGTCCTTCACCACCGACACCGTCTTCCGCGTCACCGCCACCGGGGACGGCTTCACCCTGCGGGAGGTGCCGGTGGACCCGCCCCTGACCAAGGTGTTCCCCGACGACGAATCGGACGACGAATCGGACGACGGGGAGGACGGCGACCCGGACTCCCGGACGTTCGTCGCGTACGGGGACGACGGCGACCTGGCGGGCTTCGTGGTCGTCTCGTACTCCGGCTGGAACCGCCGGCTGACCGTCGAGGACATCGAGGTCGCCCCGGAGCACCGGGGGCACGGGGTCGGGCGCGCGTTGATGGGGCTCGCGACGGAGTTCGCCCGCGAGCGGGGCGCCGGGCACCTCTGGCTGGAGGTCACCAACGTCAACGCACCGGCGATCCACGCGTACCGGCGGATGGGGTTCACCCTCTGCGGCCTGGACACCGCCCTGTACGACGGCACCGCCTCGGACGGCGAGCAGGCGCTCTACATGAGCATGCCCTGCCCCTAATCAGTACTGACAATAAAAAGATTCTTGTTTTCAAGAACTTGTCATTTGTATAGTTTTTTTATATTGTAGTTGTTCTATTTTAATCAAATGTTAGCGTGATTTATATTTTTTTTCGCCTCGACATCATCTGCCCAGATGCGAAGTTAAGTGCGCAGAAAGTAATATCATGCGTCAATCGTATGTGAATGCTGGTCGCTATACTGCTGTCGATTCGATACTAACGCCGCCATCCAGTGTCGAAAACGAGCTCTCGAGAACCCTTAAT'
BTX['need_reverse']=0 #if parts needs to be reversed:1 else:0
BTX['Primer_up_U']='GV1' #user site up
BTX['Primer_down_U']='GV2'#user site down
BTX['Primer_up']=''
BTX['Primer_up_Tm']=''
BTX['Primer_down']=''
BTX['Primer_down_Tm']=''


## CDSs ups and downs regions will be linked to KanMx to generate KO plasmids
KanMX={}
KanMX['name']='pTEF::KanMX::tTef'
KanMX['seq']='TAGGTCTAGAGATCTGTTTAGCttgcctcgtccccgccgggtcacccggccagcgacatggaggcccagaataccctccttgacagtcttgacgtgcgcagctcaggggcatgatgtgactgtcgcccgtacatttagcccatacatccccatgtataatcatttgcatccatacattttgatggccgcacggcgcgaagcaaaaattacggctcctcgctgcagacctgcgagcagggaaacgctcccctcacagacgcgttgaattgtccccacgccgcgcccctgtagagaaatataaaaggttaggatttgccactgaggttcttctttcatatacttccttttaaaatcttgctaggatacagttctcacatcacatccgaacataaacaaccatgggtaaggaaaagactcacgtttcgaggccgcgattaaattccaacatggatgctgatttatatgggtataaatgggctcgcgataatgtcgggcaatcaggtgcgacaatctatcgattgtatgggaagcccgatgcgccagagttgtttctgaaacatggcaaaggtagcgttgccaatgatgttacagatgagatggtcagactaaactggctgacggaatttatgcctcttccgaccatcaagcattttatccgtactcctgatgatgcatggttactcaccactgcgatccccggcaaaacagcattccaggtattagaagaatatcctgattcaggtgaaaatattgttgatgcgctggcagtgttcctgcgccggttgcattcgattcctgtttgtaattgtccttttaacagcgatcgcgtatttcgtctcgctcaggcgcaatcacgaatgaataacggtttggttgatgcgagtgattttgatgacgagcgtaatggctggcctgttgaacaagtctggaaagaaatgcataagcttttgccattctcaccggattcagtcgtcactcatggtgatttctcacttgataaccttatttttgacgaggggaaattaataggttgtattgatgttggacgagtcggaatcgcagaccgataccaggatcttgccatcctatggaactgcctcggtgagttttctccttcattacagaaacggctttttcaaaaatatggtattgataatcctgatatgaataaattgcagtttcatttgatgctcgatgagtttttctaatcagtactgacaataaaaagattcttgttttcaagaacttgtcatttgtatagtttttttatattgtagttgttctattttaatcaaatgttagcgtgatttatattttttttcgcctcgacatcatctgcccagatgcgaagttaagtgcgcagaaagtaatatcatgcgtcaatcgtatgtgaatgctggtcgctatactgctgtcgattcgatactaacgccgccatccagtgtcgaaaacgaGCTCTCGAGAACCCTTAAT'
KanMX['need_reverse']=0 #if parts needs to be reversed:1 else:0
KanMX['Primer_up_U']='U1' #user site up
KanMX['Primer_down_U']='U2'#user site down
KanMX['Primer_up']=''
KanMX['Primer_up_Tm']=''
KanMX['Primer_down']=''
KanMX['Primer_down_Tm']=''



##in addition,CDS will be PCRed with the forward primer of the Up and the reverse of the down
#and annealed into pCCM022 to be flanked by  HomoB and XII-dwn

KGD2_up={}
KGD2_up['name']='KGD2_up'
KGD2_up['seq']='ggcaggaccttagttttgattggctgcaaaaaaagggttgttaatatcgtctttgacggagatgcgggtggctggaattgttttgtttattattccccaagagagaagagacaagaagtagtaatgacgatggaacaaagatcggttctaataggaccacagggttgaatagagacgagtaaagacgcaacggtcaaggctttccacgagacgttcgaat'
KGD2_up['need_reverse']=1 #if parts needs to be reversed:1 else:0
KGD2_up['Primer_up_U']='GV1' #user site up
KGD2_up['Primer_down_U']='U1'#user site down
KGD2_up['Primer_up']=''
KGD2_up['Primer_up_Tm']=''
KGD2_up['Primer_down']=''
KGD2_up['Primer_down_Tm']=''

KGD2_dw={}
KGD2_dw['name']='KGD2_dw'
KGD2_dw['seq']='atgtggaaatcaaaagaatattagttgattctattatataataaatagttataatcattgtattttgatgatgaaaattcagtatctaagagttatagccaagagagtaatttttaatacttttttgcgcttttccatatctcattataaaaagaaaacattaattcatagttccatttatcatcatagcatatctttaa'
KGD2_dw['need_reverse']=1 #if parts needs to be reversed:1 else:0
KGD2_dw['Primer_up_U']='U2' #user site up
KGD2_dw['Primer_down_U']='GV2'#user site down
KGD2_dw['Primer_up']=''
KGD2_dw['Primer_up_Tm']=''
KGD2_dw['Primer_down']=''
KGD2_dw['Primer_down_Tm']=''



HEM25_up={}
HEM25_up['name']='HEM25_up'
HEM25_up['seq']='atttcacttcgtccttaatattttaacacattaactgtttgctataagcatacattgcttttaccactttttttaataatatctttttatccgtattaattaccgcatataattttagttcgaaacattaaaggtcagtcccgaagttataattataacatttaattcctgaatttaccgtattacatttaaatttgcat'
HEM25_up['need_reverse']=1 #if parts needs to be reversed:1 else:0
HEM25_up['Primer_up_U']='GV1' #user site up
HEM25_up['Primer_down_U']='U1'#user site down
HEM25_up['Primer_up']=''
HEM25_up['Primer_up_Tm']=''
HEM25_up['Primer_down']=''
HEM25_up['Primer_down_Tm']=''

HEM25_dw={}
HEM25_dw['name']='HEM25_dw'
HEM25_dw['seq']='gtgcacttatccatcgactctcttatatttatttaaacacttgtaacgcatccaatctatttcatagttagagtctctcttccttgttattgatatgatttttcttttgtaccgtgtgtgtatgtatgtatgtatgaacgtactactaagcttcaacgtcactctataaagaaactaaaaagacacgtatttatacctt'
HEM25_dw['need_reverse']=1 #if parts needs to be reversed:1 else:0
HEM25_dw['Primer_up_U']='U2' #user site up
HEM25_dw['Primer_down_U']='GV2'#user site down
HEM25_dw['Primer_up']=''
HEM25_dw['Primer_up_Tm']=''
HEM25_dw['Primer_down']=''
HEM25_dw['Primer_down_Tm']=''






VMA16_up={} #PPA1
VMA16_up['name']='VMA16_up'
VMA16_up['seq']='ggcctattgggtatctgcttgtcaggtgcaggcccaactatcttggctttggccactgagaatttcgaagaaatctctcaagaaattatcaacaggttcgccaaaaacggcatcaagtgctcctggaaactactggagcctgcctacgatggtgctagcgtcgaacagcaatgaaatgtcctttacccctagtatactgtgtt'
VMA16_up['need_reverse']=0 #if parts needs to be reversed:1 else:0
VMA16_up['Primer_up_U']='GV1' #user site up
VMA16_up['Primer_down_U']='U1'#user site down
VMA16_up['Primer_up']=''
VMA16_up['Primer_up_Tm']=''
VMA16_up['Primer_down']=''
VMA16_up['Primer_down_Tm']=''

VMA16_dw={}
VMA16_dw['name']='VMA16_dw'
VMA16_dw['seq']='agcgcatccatatttactaatagatataatgagatatctgagatacatgtgtatgtatatatgcacgttttcttttattatctaaaaatcatattatattaagtaagagaaaaaaatgtacaactatataaatatatatttatttaaaatggttttgaatttttcctattctggttgatattgcccaaaagctattcagt'
VMA16_dw['need_reverse']=0 #if parts needs to be reversed:1 else:0
VMA16_dw['Primer_up_U']='U2' #user site up
VMA16_dw['Primer_down_U']='GV2'#user site down
VMA16_dw['Primer_up']=''
VMA16_dw['Primer_up_Tm']=''
VMA16_dw['Primer_down']=''
VMA16_dw['Primer_down_Tm']=''





CAX4_up={} 
CAX4_up['name']='CAX4_up'
CAX4_up['seq']='aaccccgagaatttaaatagaacagagtatatgcatttattaacagttattacgccttgtccccagggagagtagtaagggatatgggcatttataaatgtacataagtcacgtttttgcagccaataaaatataataatagaatagtaacaaggaaagctaggccaaaactccttacatggagctagtatacccctttt'
CAX4_up['need_reverse']=1 #if parts needs to be reversed:1 else:0
CAX4_up['Primer_up_U']='GV1' #user site up
CAX4_up['Primer_down_U']='U1'#user site down
CAX4_up['Primer_up']=''
CAX4_up['Primer_up_Tm']=''
CAX4_up['Primer_down']=''
CAX4_up['Primer_down_Tm']=''

CAX4_dw={}
CAX4_dw['name']='CAX4_dw'
CAX4_dw['seq']='aagtgtgggtgacatgctaaatccgcggctttccttaagctttttcacaaaccatctattatgtccagaagctccgctgattatccgtggaaaagggtcttattctgccgtaggaaacttcttctcgagaaaaaagtgagttcacccgcggcgctctcgtaaaacccgagggccgattctttccctccgtggaacaatcg'
CAX4_dw['need_reverse']=1 #if parts needs to be reversed:1 else:0
CAX4_dw['Primer_up_U']='U2' #user site up
CAX4_dw['Primer_down_U']='GV2'#user site down
CAX4_dw['Primer_up']=''
CAX4_dw['Primer_up_Tm']=''
CAX4_dw['Primer_down']=''
CAX4_dw['Primer_down_Tm']=''



PRO2_up={} #PPA1
PRO2_up['name']='PRO2_up'
PRO2_up['seq']='ttgtaaagtgaccttgttttgactttttaacgagtaaaaaaacgagttaacaaattcttgacgtaaacgataaagccgcccatcaaaagaaatgcaataatgtcaatgatgaaatacctatacaggtagatggatttcgtgttgttcttcgatttatttaacacgtttttcagctctatcaaactatccatttcccttag'
PRO2_up['need_reverse']=1 #if parts needs to be reversed:1 else:0
PRO2_up['Primer_up_U']='GV1' #user site up
PRO2_up['Primer_down_U']='U1'#user site down
PRO2_up['Primer_up']=''
PRO2_up['Primer_up_Tm']=''
PRO2_up['Primer_down']=''
PRO2_up['Primer_down_Tm']=''

PRO2_dw={}
PRO2_dw['name']='PRO2_dw'
PRO2_dw['seq']='gagtagatgtaagacgtgaaaatgccatggtaaaacgtattgtttagtgaaatgtatactaagtaggagtaaaaatcagcaatttctggtgttgaaatgtgaacaacagcgttctcctccgaaaaagcaagctataacgttgctcaacttttgattccgaaatcagaagcttaaataaggaaaactgaaagctaattgga'
PRO2_dw['need_reverse']=1 #if parts needs to be reversed:1 else:0
PRO2_dw['Primer_up_U']='U2' #user site up
PRO2_dw['Primer_down_U']='GV2'#user site down
PRO2_dw['Primer_up']=''
PRO2_dw['Primer_up_Tm']=''
PRO2_dw['Primer_down']=''
PRO2_dw['Primer_down_Tm']=''



GLR1_up={} #PPA1
GLR1_up['name']='GLR1_up'
GLR1_up['seq']='tggtcaattctatgccttttatgcacattgcatgagtattctaaaaagatgctgcatgctgcccgtaaatcttcattattttcagagtcaggtacggaaaagacgacagtttctccgtataacagcattgaaagcgtggaagaatcaaactcggctctagattttacgcgtttagcataaaattgttggttttatacgta'
GLR1_up['need_reverse']=0 #if parts needs to be reversed:1 else:0
GLR1_up['Primer_up_U']='GV1' #user site up
GLR1_up['Primer_down_U']='U1'#user site down
GLR1_up['Primer_up']=''
GLR1_up['Primer_up_Tm']=''
GLR1_up['Primer_down']=''
GLR1_up['Primer_down_Tm']=''

GLR1_dw={}
GLR1_dw['name']='GLR1_dw'
GLR1_dw['seq']='cttttactgctttctaactttaagccttcaaagaagaagctcttctctttctgatttcagccttctcagcctttctttcagacaatctcttagccaacaattgagcgtattcggcagcagcttctctttgagcttgagcgtttctgaccttcaaagctctttggtgtctctttctttgcaatctttgaggagtaaccaatc'
GLR1_dw['need_reverse']=0 #if parts needs to be reversed:1 else:0
GLR1_dw['Primer_up_U']='U2' #user site up
GLR1_dw['Primer_down_U']='GV2'#user site down
GLR1_dw['Primer_up']=''
GLR1_dw['Primer_up_Tm']=''
GLR1_dw['Primer_down']=''
GLR1_dw['Primer_down_Tm']=''



YSY6_up={} 
YSY6_up['name']='YSY6_up'
YSY6_up['seq']='ggagtcttcttgcgagaaaagaaataattataatataaaaaaaaattgaatggagataagaagtttaactagcaaagttcgagagatataaaaagtagagatgcgaaaataaaattaaaacaaatctgaaaataatggaagtgaaacaaggctaagcaagaaaattttctatattgccaagacaggaacgtttgtgatga'
YSY6_up['need_reverse']=0 #if parts needs to be reversed:1 else:0
YSY6_up['Primer_up_U']='GV1' #user site up
YSY6_up['Primer_down_U']='U1'#user site down
YSY6_up['Primer_up']=''
YSY6_up['Primer_up_Tm']=''
YSY6_up['Primer_down']=''
YSY6_up['Primer_down_Tm']=''

YSY6_dw={}
YSY6_dw['name']='YSY6_dw'
YSY6_dw['seq']='gcagtattacccgcgttgtcatttatggatgcgaaaaaagatgaagtaattcaagtgctattgtgatttaccatgccaatcaaacgtgcaacaaagatgagtatgaggggggacccaggcatagtacacagaaaagctttgtagactgtctttgttttcttaatttttcaagtaggctttctgcttttcttctgtactta'
YSY6_dw['need_reverse']=0 #if parts needs to be reversed:1 else:0
YSY6_dw['Primer_up_U']='U2' #user site up
YSY6_dw['Primer_down_U']='GV2'#user site down
YSY6_dw['Primer_up']=''
YSY6_dw['Primer_up_Tm']=''
YSY6_dw['Primer_down']=''
YSY6_dw['Primer_down_Tm']=''





PDX3_up={} 
PDX3_up['name']='PDX3_up'
PDX3_up['seq']='aggtagagtatctttaaaatatctaatcatgcaaattagtcatttcttctttcttataagttgcggccccttctagtatttacaatttacccggcgctggttctggtggcatctataagagctctatctggctgctaagcagacttgctgttacttgtatgtgctgtttatcaagattggggcattacgcatgtgtatgt'
PDX3_up['need_reverse']=1 #if parts needs to be reversed:1 else:0
PDX3_up['Primer_up_U']='GV1' #user site up
PDX3_up['Primer_down_U']='U1'#user site down
PDX3_up['Primer_up']=''
PDX3_up['Primer_up_Tm']=''
PDX3_up['Primer_down']=''
PDX3_up['Primer_down_Tm']=''

PDX3_dw={}
PDX3_dw['name']='PDX3_dw'
PDX3_dw['seq']='tctttggatgaatttgtcggttctaactcttttttggaaacggcaaggtgagcagtaggcagctcatctcttcaactatttttataacatataatttccctttgtgaaaaaattttcccgccatttccaaatttcggaattcctttttcttttttttcatctcggtatattctgacatattaccctataggcggtataaa'
PDX3_dw['need_reverse']= 1 #if parts needs to be reversed:1 else:0
PDX3_dw['Primer_up_U']='U2' #user site up
PDX3_dw['Primer_down_U']='GV2'#user site down
PDX3_dw['Primer_up']=''
PDX3_dw['Primer_up_Tm']=''
PDX3_dw['Primer_down']=''
PDX3_dw['Primer_down_Tm']=''



Parts=[KanMX,KGD2_up,KGD2_dw,HEM25_up,HEM25_dw,VMA16_up,VMA16_dw,
       CAX4_up,CAX4_dw,PRO2_up,PRO2_dw,GLR1_up,GLR1_dw,YSY6_up,YSY6_dw,PDX3_up,PDX3_dw]


In [111]:
# Fit and Yellow candidate
#PRY1
#GET4
#YLR271W 
#YCR101C
#STV1

PRY1_up={} 
PRY1_up['name']='PRY1_up'
PRY1_up['seq']='ttctagtatctacactgaagtcaacgccgcaactatcattaaccccggagaaacatcttctctcgctagcgatttcgccacatctgaaaagccaaacgagcccacttctgtcaaatccacctcaaacgaaggcacctcttccacaacaacaacctaccaacagactgttgctacactgtatgccaagccctccagcacaa'
PRY1_up['need_reverse']=0 #if parts needs to be reversed:1 else:0
PRY1_up['Primer_up_U']='GV1' #user site up
PRY1_up['Primer_down_U']='U1'#user site down
PRY1_up['Primer_up']=''
PRY1_up['Primer_up_Tm']=''
PRY1_up['Primer_down']=''
PRY1_up['Primer_down_Tm']=''

PRY1_dw={} 
PRY1_dw['name']='PRY1_dw'
PRY1_dw['seq']='gaaaaattttcgctcattttcgcgaattgttgttggcgacactcgacaatttcaatttaatgtagctcttaaagacttgcaacggatagtatatttactaatttatagtttttacatttcgtacagaacttccttcgaaacattggctattagcttttcaattgaacgtctttgaaaaagaacgtctaagtacacaacagca'
PRY1_dw['need_reverse']=0 #if parts needs to be reversed:1 else:0
PRY1_dw['Primer_up_U']='U2' #user site up
PRY1_dw['Primer_down_U']='GV2'#user site down
PRY1_dw['Primer_up']=''
PRY1_dw['Primer_up_Tm']=''
PRY1_dw['Primer_down']=''
PRY1_dw['Primer_down_Tm']=''

GET4_up={} 
GET4_up['name']='GET4_up'
GET4_up['seq']='aaagtatgtcaaccgtcgagtgagttagtagcccgtttgttgaaataagtcaataattatttaccatcaaaatacacctttcagtattttgttcgttttgtcgctcttcgttaaggtaacaaacaatgggaaaaaaagtgaaaggaaagtcaaaaaaatgaaataataacatgtgcggtaaaacaaccaagtaaacatca'
GET4_up['need_reverse']= 0#if parts needs to be reversed:1 else:0
GET4_up['Primer_up_U']='GV1' #user site up
GET4_up['Primer_down_U']='U1'#user site down
GET4_up['Primer_up']=''
GET4_up['Primer_up_Tm']=''
GET4_up['Primer_down']=''
GET4_up['Primer_down_Tm']=''

GET4_dw={} 
GET4_dw['name']='GET4_dw'
GET4_dw['seq']='aaagagcatatttgcgaaggaatagataaatatgtttgcggtggcttgacgaccatgacgtatatacatatatatgtatgtgctagctatttgtcgtctttaatgatagcagacctattaagggcctccaacaattcaggcctcttggcgtctatcaagttttgtttagcttctgtataagagtatagctttctatgtct' #
GET4_dw['need_reverse']=0 #if parts needs to be reversed:1 else:0
GET4_dw['Primer_up_U']='U2' #user site up
GET4_dw['Primer_down_U']='GV2'#user site down
GET4_dw['Primer_up']=''
GET4_dw['Primer_up_Tm']=''
GET4_dw['Primer_down']=''
GET4_dw['Primer_down_Tm']=''





CMG1_up={} 
CMG1_up['name']='CMG1_up'
CMG1_up['seq']='ggttgttaaaattctcacatcatccacctttttcctactctcattttctttttttgaaagttgtttgttttatataccttagtttcacaacaacataaggcatgcctatttgagattttgtgattaaaaaataatagaatacgtactatataaccgtcggtttaaagtaaataaattaaagactaagaatgaaacaatta'
CMG1_up['need_reverse']=0 #if parts needs to be reversed:1 else:0
CMG1_up['Primer_up_U']='GV1' #user site up
CMG1_up['Primer_down_U']='U1'#user site down
CMG1_up['Primer_up']=''
CMG1_up['Primer_up_Tm']=''
CMG1_up['Primer_down']=''
CMG1_up['Primer_down_Tm']=''


CMG1_dw={} 
CMG1_dw['name']='CMG1_dw'
CMG1_dw['seq']='gatgattttcttgaaactctccttacccaataaatcatctgacgataaattactaaagatatcgatgaatccattataaatggcgttgtccttggatgctaactcggtgaaaaataatctgcacatatcacttattccctggtctggattgtctaggcatttggccatttctcctaattgtcctttaaccttaacctgtc'
CMG1_dw['need_reverse']=0
CMG1_dw['Primer_up_U']='U2' #user site up
CMG1_dw['Primer_down_U']='GV2'#user site down
CMG1_dw['Primer_up']=''
CMG1_dw['Primer_up_Tm']=''
CMG1_dw['Primer_down']=''
CMG1_dw['Primer_down_Tm']=''

YCR101C_up={}
YCR101C_up['name']='YCR101C_up'
YCR101C_up['seq']='tacctataatagcggctcgctgtccgaacgattgcgccgcacttttctgcctcttacattcccttttcctaaggcgataagcaggctgaatggaaactacgatataacgcagtagaggtcccccaagagctggaaagaaaatattggtttgatagccagcactattctatccagtgggctcttgtaccattttctgtggt'
YCR101C_up['need_reverse']=0 #if parts needs to be reversed:1 else:0
YCR101C_up['Primer_up_U']='GV1' #user site up
YCR101C_up['Primer_down_U']='U1'#user site down
YCR101C_up['Primer_up']=''
YCR101C_up['Primer_up_Tm']=''
YCR101C_up['Primer_down']=''
YCR101C_up['Primer_down_Tm']=''


YCR101C_dw={} 
YCR101C_dw['name']='YCR101C_dw'
YCR101C_dw['seq']='aatgatacttcgatactctgtttgttctctaaccgtgaatacgatgaagaatatcgcttaactgctacctacacacagttggctttcagtagtgatggaggcaaaacgtttaaagattttgatgaatttaaagataagattgttactcaatacaagatattgaagtcatatgtggtcgttttaacgcaagatgacgggca'
YCR101C_dw['need_reverse']=0
YCR101C_dw['Primer_up_U']='U2' #user site up
YCR101C_dw['Primer_down_U']='GV2'#user site down
YCR101C_dw['Primer_up']=''
YCR101C_dw['Primer_up_Tm']=''
YCR101C_dw['Primer_down']=''
YCR101C_dw['Primer_down_Tm']=''

STV1_up={}
STV1_up['name']='STV1_up'
STV1_up['seq']='tattaataggtgcttcttccagggtctagttcattacctcccctaaatatcattctcagatcattatagcttctatttcctgttatgcgggtagcggggcttatttttgtttctcaaaatagaggtgtccttttttgccatttttgtttatgtagtcaactgcataggtagttaagaaaagacataggcccacgaaggtg'
STV1_up['need_reverse']=0 #if parts needs to be reversed:1 else:0
STV1_up['Primer_up_U']='GV1' #user site up
STV1_up['Primer_down_U']='U1'#user site down
STV1_up['Primer_up']=''
STV1_up['Primer_up_Tm']=''
STV1_up['Primer_down']=''
STV1_up['Primer_down_Tm']=''


STV1_dw={} 
STV1_dw['name']='STV1_dw'
STV1_dw['seq']='cacatagtttacggtatatatatgtctgggtcggtcaagtggtctaccaataggtcataaatttgagcagggatcttagcgatgaacccgacgcccagccgaatgatctcgtcagcatcaaaatccggaaactgtcgtagtaaattaacaggggaatccgatttgaaaaccttggatctcattttaactaggaatgccac'
STV1_dw['need_reverse']=0
STV1_dw['Primer_up_U']='U2' #user site up
STV1_dw['Primer_down_U']='GV2'#user site down
STV1_dw['Primer_up']=''
STV1_dw['Primer_up_Tm']=''
STV1_dw['Primer_down']=''
STV1_dw['Primer_down_Tm']=''


Parts=[PRY1_up,PRY1_dw,GET4_up,GET4_dw,CMG1_up,CMG1_dw,YCR101C_up,YCR101C_dw,STV1_up,STV1_dw]


In [7]:
len(ARO2_up['seq'])

808

In [12]:


ARO2_up={} 
ARO2_up['name']='ARO2_up'
ARO2_up['seq']='gctcttctatatgtcttaagccttgctttgctaatatggacaggatcggtagttactcttctcttgttacttttcttttgcctcttcctcttgttcttctcactccactttttttgttttacccgtgaacacgtacactatacgctcccacccaaatgccactcacttaaattccaattcgattcgattccgagttcgtc'
ARO2_up['need_reverse']=0
ARO2_up['Primer_up_U']='GV1' #user site up
ARO2_up['Primer_down_U']='U1'#user site down
ARO2_up['Primer_up']=''
ARO2_up['Primer_up_Tm']=''
ARO2_up['Primer_down']=''
ARO2_up['Primer_down_Tm']=''

ARO2_dw={} 
ARO2_dw['name']='ARO2_dw'
ARO2_dw['seq']='atattatatgttagatatacaaagttttttatatgtaattattttttctgctactttaaagaaaatgtcacaaaatcaaataaaaagcttataaatcttcagtaataaaacccttgtgagaaacgtagataccgtccaaaaatttacggatatccttgtttctgacacgacagatttgttgcaagtcagcagcgttttgg'
ARO2_dw['need_reverse']=0
ARO2_dw['Primer_up_U']='U2' #user site up
ARO2_dw['Primer_down_U']='GV2'#user site down
ARO2_dw['Primer_up']=''
ARO2_dw['Primer_up_Tm']=''
ARO2_dw['Primer_down']=''
ARO2_dw['Primer_down_Tm']=''

Parts=[ARO2_dw,ARO2_up]


In [13]:
#fills the part dicts with primers:

dt.User_Assebmly(Parts, 50, User_Overhangs)




Will use input Tm_target for design:  50


[{'name': 'ARO2_dw',
  'seq': 'ATATTATATGTTAGATATACAAAGTTTTTTATATGTAATTATTTTTTCTGCTACTTTAAAGAAAATGTCACAAAATCAAATAAAAAGCTTATAAATCTTCAGTAATAAAACCCTTGTGAGAAACGTAGATACCGTCCAAAAATTTACGGATATCCTTGTTTCTGACACGACAGATTTGTTGCAAGTCAGCAGCGTTTTGG',
  'need_reverse': 0,
  'Primer_up_U': 'U2',
  'Primer_down_U': 'GV2',
  'Primer_up': 'ACACAGGCuATATTATATGTTAGATATACAAAGTTTTTTATATG',
  'Primer_up_Tm': 50.504044345073794,
  'Primer_down': 'CACGCGAuCCAAAACGCTGCTGACT',
  'Primer_down_Tm': 50.33839887054705},
 {'name': 'ARO2_up',
  'seq': 'GCTCTTCTATATGTCTTAAGCCTTGCTTTGCTAATATGGACAGGATCGGTAGTTACTCTTCTCTTGTTACTTTTCTTTTGCCTCTTCCTCTTGTTCTTCTCACTCCACTTTTTTTGTTTTACCCGTGAACACGTACACTATACGCTCCCACCCAAATGCCACTCACTTAAATTCCAATTCGATTCGATTCCGAGTTCGTC',
  'need_reverse': 0,
  'Primer_up_U': 'GV1',
  'Primer_down_U': 'U1',
  'Primer_up': 'CGTGCGAuGCTCTTCTATATGTCTTAAGCC',
  'Primer_up_Tm': 50.28212878499204,
  'Primer_down': 'AAGAGGGCuGACGAACTCGGAATCGAAT',
  'Primer_down_Tm': 50.08954894143767}]

In [14]:



primer_name=[]
primer_dir=[]
primer_tm=[]
primer_OH=[]
primer_template_name=[]
primer_seq=[]
primer_template_len=[]
benchling=[]
i=108

for part in Parts:
    #here we adapt the order with which the overhang names are appended to match the Overhang names to the inversion of sequences
    if part['need_reverse']==0:
        primer_OH.append(part['Primer_up' +'_U'])
        primer_OH.append(part['Primer_down'+'_U'])
    elif part['need_reverse']==1:
        primer_OH.append(part['Primer_down'+'_U'])
        primer_OH.append(part['Primer_up' +'_U'])

    for primer in ['Primer_up', 'Primer_down']:
        primer_name.append('BTX.1')#.zfill fills primer number wiht leading 0
        primer_seq.append(part[primer])
        primer_tm.append(round(part[primer +'_Tm'],2))
        primer_template_len.append(len(part['seq']))
        primer_template_name.append(part['name'])
        #primer_OH.append(part[primer +'_U'])
        primer_dir.append('fwd'  if primer[7:] == 'up' else 'rvs' )
        benchling.append(part[primer].replace('u','t'))
        i+=1
## need to fix the current overhang name bug (if seq needs inversion the OH name doenst match)
        


df = np.stack([primer_name, primer_seq,primer_dir, primer_tm,primer_OH, primer_template_name, primer_template_len, benchling] ).T
Table=pd.DataFrame(df , columns=['Primer Name', 'Seq','Dir', 'Tm (c)','User', 'Template', 'Template_Len','benchling'])

Table.set_index('Primer Name')

,Seq,Dir,Tm (c),User,Template,Template_Len,benchling
Primer Name,,,,,,,
BTX.1,ACACAGGCuATATTATATGTTAGATATACAAAGTTTTTTATATG,fwd,50.5,U2,ARO2_dw,200,ACACAGGCtATATTATATGTTAGATATACAAAGTTTTTTATATG
BTX.1,CACGCGAuCCAAAACGCTGCTGACT,rvs,50.34,GV2,ARO2_dw,200,CACGCGAtCCAAAACGCTGCTGACT
BTX.1,CGTGCGAuGCTCTTCTATATGTCTTAAGCC,fwd,50.28,GV1,ARO2_up,200,CGTGCGAtGCTCTTCTATATGTCTTAAGCC
BTX.1,AAGAGGGCuGACGAACTCGGAATCGAAT,rvs,50.09,U1,ARO2_up,200,AAGAGGGCtGACGAACTCGGAATCGAAT


# Design upstream and downstream confirmation primers 

In [128]:

genes = ['cax4','glr1','hem25','kgd2','pdx3','pro2','vma16','ysy6']

path2parts = '/Users/paucac/Documents/PhD/CAB/CRISPA/Betaxanthin/DNA_parts/Overexpression/'

#store file name
part_file={}
for gene in genes:
    part_file[gene]= gene+".gb"
part_seq={}
for gene in genes:
    recs = [rec for rec in SeqIO.parse(path2parts + part_file[gene], "genbank")]
    part_seq[gene] = str(recs[0].seq)

In [16]:
os.listdir('/Users/paucac/Documents/PhD/CAB/CRISPA/CCM/DNA_parts')

['.DS_Store', 'KO_confirmations']

In [18]:
genes = ['aro2']

path2parts = '/Users/paucac/Documents/PhD/CAB/CRISPA/CCM/DNA_parts/KO_confirmations/'

#store file name
part_file={}
for gene in genes:
    part_file[gene]= gene+".gb"
part_seq={}
for gene in genes:
    recs = [rec for rec in SeqIO.parse(path2parts + part_file[gene], "genbank")]
    part_seq[gene] = str(recs[0].seq)

In [115]:
genes = ['pry1','get4','cmg1','ycr101c','stv1']

path2parts = '/Users/paucac/Documents/PhD/CAB/CRISPA/Betaxanthin/DNA_parts/KO_confirmations/second_batch/'
os.listdir(path2parts)
part_file={}
for gene in genes:
    part_file[gene]= gene+".gb"
    
part_seq={}
for gene in genes:
    recs = [rec for rec in SeqIO.parse(path2parts + part_file[gene], "genbank")]
    part_seq[gene] = str(recs[0].seq)

In [125]:
Table['Primer Name']=['BTX.'+str(s) for s in range(110,144)]

In [20]:

primer_name=[]
primer_dir=[]
primer_tm=[]
primer_OH=[]
primer_template_name=[]
primer_seq=[]
primer_template_len=[]
benchling=[]

for gene in genes:
    
    #find user primer sequence in table:
    i_primer_up = (Table['Template'].str.contains(gene.upper()) ) & ( Table['User'].str.contains('GV1') )
    i_primer_dw = (Table['Template'].str.contains(gene.upper()) ) & ( Table['User'].str.contains('GV2') )
    
    primer_up =  Table[i_primer_up]['Seq'].values[0]
    primer_dw =  Table[i_primer_dw]['Seq'].values[0]

    #remove user overhang
    primer_up=  primer_up[primer_up.find('u')+1:]
    primer_dw=  primer_dw[primer_dw.find('u')+1:]
    #find the upper position of the user primers
    #if the gene sequence used was in the reverse direction the up primer wont be found 
    #in that case the position of the down primers will give the upper position
    
    pos_up = part_seq[gene].find(primer_up)
    if pos_up == -1:
        #if primer up is not found it is because sequence of the gene was in the reverse orientation
        #primer_up will therefore be downstream of the sequence
        pos_dw = part_seq[gene].find(dt.reverse_complement(primer_up))#to find reverse primer we need to search its unreversed complement
        pos_up = part_seq[gene].find(primer_dw)
    else :
        pos_dw = part_seq[gene].find(dt.reverse_complement(primer_dw))#to find reverse primer we need to search its unreversed complement
        

    print(gene.upper()+'_CDS,'+ str(round((pos_dw- pos_up)/1000,2)) +'kb,'+ Table[i_primer_up]['Primer Name'].item(),',', Table[i_primer_dw]['Primer Name'].item() )
    
    valid_up = dt.make_anneal(part_seq[gene][pos_up-150:pos_up], 'F', 50 )
    
    valid_dw = dt.make_anneal(part_seq[gene][pos_dw+150:pos_dw+250], 'R', 50 )
    primer_name.append('BTX.')
    primer_name.append('BTX.')
    primer_dir.append('fw')
    primer_dir.append('rv')
    
    primer_tm.append(round(valid_up[1],2))
    primer_tm.append(round(valid_dw[1],2))
    primer_seq.append(valid_up[0])
    primer_seq.append(valid_dw[0])
    primer_OH.append('up.out')
    primer_OH.append('dw.out')
    
    primer_template_len.append(pos_dw+250 -(pos_up-150))
    primer_template_len.append(pos_dw+250 -(pos_up-150))
    primer_template_name.append(gene)
    primer_template_name.append(gene)

    
    
df = np.stack([primer_name, primer_seq,primer_dir, primer_tm, primer_OH,primer_template_name, primer_template_len] ).T
Confirmation_Primers=pd.DataFrame(df , columns=['Primer Name', 'Seq','Dir', 'Tm (c)','position', 'Template', 'Template_Len'])

Confirmation_Primers.set_index('Primer Name')

ARO2_CDS,2.27kb,BTX.1 , BTX.1


,Seq,Dir,Tm (c),position,Template,Template_Len
Primer Name,,,,,,
BTX.,TGTTTCTTATCTAGCACTCTGG,fw,51.05,up.out,aro2,2665
BTX.,ATGATCACTGGTGTCACCA,rv,51.32,dw.out,aro2,2665


In [13]:
Primer_Table

,Primer Name,Seq,Dir,Tm (c),position,Template,Template_Len
0,BTX.,AACAATCTCGACATCTTGTAGC,fw,51.24,up.out,pry1,2778
1,BTX.,GGCGAATCAATGGTAGTTAC,rv,50.86,dw.out,pry1,2778
2,BTX.,CCACTCGATTGAGAACCG,fw,50.88,up.out,get4,1742
3,BTX.,CCCAACGAGAAACTTGGG,rv,52.51,dw.out,get4,1742
4,BTX.,GACCAGAAGGCTACATGAA,fw,50.25,up.out,cmg1,2209
5,BTX.,TTATCATCAAAATATTGTGAAAAGAGC,rv,50.46,dw.out,cmg1,2209
6,BTX.,TATATCCTTACTATCTACTATCCAAC,fw,50.7,up.out,ycr101c,2185
7,BTX.,CATTCAACAGGTGAGAATTTCAA,rv,50.22,dw.out,ycr101c,2185
8,BTX.,TTTGCATCTTTTTGTCCACG,fw,50.16,up.out,stv1,3765
9,BTX.,ACTAAGAACCTAAACGGCG,rv,51.74,dw.out,stv1,3765


In [75]:
Table

,Primer Name,Seq,Dir,Tm (c),User,Template,Template_Len,benchling
0,BTX.1,AGCCCTCTuTAGGTCTAGAGATCTGTTTAGC,fwd,50.27,U1,pTEF::KanMX::tTef,1473,AGCCCTCTtTAGGTCTAGAGATCTGTTTAGC
1,BTX.1,AGCCTGTGuATTAAGGGTTCTCGAGAGC,rvs,50.05,U2,pTEF::KanMX::tTef,1473,AGCCTGTGtATTAAGGGTTCTCGAGAGC
2,BTX.1,AAGAGGGCuGGCAGGACCTTAGTTTTGA,fwd,51.38,U1,KGD2_up,220,AAGAGGGCtGGCAGGACCTTAGTTTTGA
3,BTX.1,CGTGCGAuATTCGAACGTCTCGTGGA,rvs,50.93,GV1,KGD2_up,220,CGTGCGAtATTCGAACGTCTCGTGGA
4,BTX.1,CACGCGAuATGTGGAAATCAAAAGAATATTAGTTG,fwd,50.6,GV2,KGD2_dw,200,CACGCGAtATGTGGAAATCAAAAGAATATTAGTTG
5,BTX.1,ACACAGGCuTTAAAGATATGCTATGATGATAAATGG,rvs,50.39,U2,KGD2_dw,200,ACACAGGCtTTAAAGATATGCTATGATGATAAATGG
6,BTX.1,AAGAGGGCuATTTCACTTCGTCCTTAATATTTTAAC,fwd,50.91,U1,HEM25_up,200,AAGAGGGCtATTTCACTTCGTCCTTAATATTTTAAC
7,BTX.1,CGTGCGAuATGCAAATTTAAATGTAATACGGTA,rvs,50.4,GV1,HEM25_up,200,CGTGCGAtATGCAAATTTAAATGTAATACGGTA
8,BTX.1,CACGCGAuGTGCACTTATCCATCGACT,fwd,50.22,GV2,HEM25_dw,199,CACGCGAtGTGCACTTATCCATCGACT
9,BTX.1,ACACAGGCuAAGGTATAAATACGTGTCTTTTTAGT,rvs,51.35,U2,HEM25_dw,199,ACACAGGCtAAGGTATAAATACGTGTCTTTTTAGT


## Format a PCR table for USER primers

In [81]:


#create a PCR table
tube_label = []
template = []
tm=[]
Fwd = []
Rev = []
Len = []
Primer_Table = Table.copy()
#CRITICAL: edit name to match the name as ordered:
Primer_Table['Primer Name'] = ['BTX.'+str(n) for n in range(161, 161 + len(Primer_Table))]
genes = ['pry1','get4','cmg1','ycr101c','stv1']
for i, gene in enumerate(genes):
    
    gene = gene.upper()
    #upstream PCR
    template.append(gene)
    tm.append('60C')
    i = (Primer_Table['Template']== gene +'_up')  & (Primer_Table['Dir']== 'fwd')
    Fwd.append(Primer_Table['Primer Name'][i].values[0])
    
    i = (Primer_Table['Template']==  gene +'_up') & (Primer_Table['Dir']== 'rvs')
    Rev.append(Primer_Table['Primer Name'][i].values[0])
    tube_label.append(gene +'.up')
    l = str(round(int(Primer_Table['Template_Len'][i].values[0])/1000,2)) +'kb'
    Len.append(l)
    #downstream PCR
    
    template.append(gene)
    tm.append('60C')
    i = (Primer_Table['Template']== gene +'_dw')  & (Primer_Table['Dir']== 'fwd')
    Fwd.append(Primer_Table['Primer Name'][i].values[0])
    i = (Primer_Table['Template']== gene +'_dw') & (Primer_Table['Dir']== 'rvs')
    Rev.append(Primer_Table['Primer Name'][i].values[0])
    tube_label.append(gene +'.dw')
    l = str(round(int(Primer_Table['Template_Len'][i].values[0])/1000,2)) +'kb'
    Len.append(l)
    
df = np.stack([ template, tube_label,tm , Len, Fwd, Rev ] ).T
PCR_Table=pd.DataFrame(df , columns=[  'Template','Tube_label', 'tm','Size', 'Fw','Rv'])

PCR_Table.set_index('Template')


#print(PCR_Table.to_string(index=0))
print(PCR_Table.to_csv(sep=',', index=False))

Template,Tube_label,tm,Size,Fw,Rv
PRY1,PRY1.up,60C,0.2kb,BTX.161,BTX.162
PRY1,PRY1.dw,60C,0.2kb,BTX.163,BTX.164
GET4,GET4.up,60C,0.2kb,BTX.165,BTX.166
GET4,GET4.dw,60C,0.2kb,BTX.167,BTX.168
CMG1,CMG1.up,60C,0.2kb,BTX.169,BTX.170
CMG1,CMG1.dw,60C,0.2kb,BTX.171,BTX.172
YCR101C,YCR101C.up,60C,0.2kb,BTX.173,BTX.174
YCR101C,YCR101C.dw,60C,0.2kb,BTX.175,BTX.176
STV1,STV1.up,60C,0.2kb,BTX.177,BTX.178
STV1,STV1.dw,60C,0.2kb,BTX.179,BTX.180



In [64]:
print(PCR_Table)

   Template Tube_label   tm    Size       Fw       Rv
0      CAX4    CAX4.up  60C   0.2kb  BTX.124  BTX.125
1      CAX4    CAX4.dw  60C   0.2kb  BTX.126  BTX.127
2      GLR1    GLR1.up  60C   0.2kb  BTX.132  BTX.133
3      GLR1    GLR1.dw  60C   0.2kb  BTX.134  BTX.135
4     HEM25   HEM25.up  60C   0.2kb  BTX.116  BTX.117
5     HEM25   HEM25.dw  60C   0.2kb  BTX.118  BTX.119
6      KGD2    KGD2.up  60C  0.22kb  BTX.112  BTX.113
7      KGD2    KGD2.dw  60C   0.2kb  BTX.114  BTX.115
8      PDX3    PDX3.up  60C   0.2kb  BTX.136  BTX.137
9      PDX3    PDX3.dw  60C   0.2kb  BTX.138  BTX.139
10     PRO2    PRO2.up  60C   0.2kb  BTX.128  BTX.129
11     PRO2    PRO2.dw  60C   0.2kb  BTX.130  BTX.131
12    VMA16   VMA16.up  60C   0.2kb  BTX.120  BTX.121
13    VMA16   VMA16.dw  60C   0.2kb  BTX.122  BTX.123
14     YSY6    YSY6.up  60C   0.2kb  BTX.140  BTX.141
15     YSY6    YSY6.dw  60C   0.2kb  BTX.142  BTX.143


In [54]:
## Format a PCR table

#create a PCR table
tube_label = []
template = []
tm=[]
Fwd = []
Rev = []
Len = []
Primer_Table = Confirmation_Primers.copy()
for i, gene in enumerate(genes):
    
    #upstream PCR
    template.append(gene)
    tm.append('60C')
    i = (Primer_Table['Template']== gene) & (Primer_Table['Dir']== 'fw')
    Fwd.append(Primer_Table['Primer Name'][i].values[0])
    i = (Primer_Table['Template']== gene) & (Primer_Table['Dir']== 'rv')
    Rev.append(Primer_Table['Primer Name'][i].values[0])
    tube_label.append(gene +'.up')
    Len.append(Primer_Table['Template_Len'][i].values[0])
    #downstream PCR
    
    template.append(gene)
    tm.append('60C')
    i = (Primer_Table['Template']== gene) & (Primer_Table['Dir']== 'fw')
    Fwd.append(Primer_Table['Primer Name'][i].values[0])
    i = (Primer_Table['Template']== gene) & (Primer_Table['Dir']== 'rv')
    Rev.append(Primer_Table['Primer Name'][i].values[0])
    tube_label.append(gene +'.dw')
    Len.append(Primer_Table['Template_Len'][i].values[0])
    
df = np.stack([ tube_label, template,tm , Len, Fwd, Rev ] ).T
PCR_Table=pd.DataFrame(df , columns=[  'Tube_label','Template', 'tm','Product (bp)', 'Fwd','Rev'])

PCR_Table.set_index('Template')


PCR_Table

IndexError: index 0 is out of bounds for axis 0 with size 0